<a href="https://colab.research.google.com/github/Svanzi/Jazz-Improvisation/blob/main/Jazz_Improv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import re
import numpy as np
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from tensorflow.keras.utils import to_categorical
# import play
import os

In [2]:
!git clone https://github.com/Svanzi/Jazz-Improvisation.git
%cd Jazz-Improvisation

Cloning into 'Jazz-Improvisation'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 52 (delta 13), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 48.93 KiB | 4.08 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/Jazz-Improvisation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from music21 import converter, instrument, note, chord, stream

In [ ]:
songs = glob("MIDI/*.mid")
# Per verificare che il caricamento dei file sia andato a buon fine e che i file midi siano presenti
print(songs)

[]


## Load all the notes in an array

In [ ]:
def get_notes():
  notes = []
  for file in songs:
    midi = converter.parse(file)
    notes_to_parse = []

    try:
      parts = instrument.partitionByInstrument(midi)
    except:
      pass
    if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in  notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

  path = '/content/drive/My Drive/University/AI2Arts/data'
  os.makedirs(path, exist_ok=True)

  with open(os.path.join(path, 'notes'), 'wb') as filepath:
    pickle.dump(notes, filepath)

  return notes

## Define the input and output sequences

In [ ]:
# n_vocab represent the total number of unique notes in the variable 'notes_data'
def sequences(notes, n_vocab):
  sequence_length = 100

  pitchnames = sorted(set(item for item in notes))

  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []

  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

  network_input = network_input / float(n_vocab)

  network_output = to_categorical(network_output)

  return (network_input, network_output)

## Create the Network

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Flatten

def create_network(network_input, n_vocab):
  """Create the model architecture"""
  model = Sequential()
  model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

  return model

## Train

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

def train(model, network_input, network_output, epochs):
  """Train the neural network"""

  #filepath = "weights.best.music3.epoch-{epoch:02d}.keras"
  filepath = "/content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras"

  checkpoint = ModelCheckpoint(
      filepath,
      monitor='loss',
      verbose=2,
      save_best_only=True,
      mode='min'
  )

  early_stop = EarlyStopping(
    monitor='loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
  )

  model.fit(network_input, network_output, epochs=epochs, batch_size=64, callbacks = [checkpoint, early_stop])

In [ ]:
def train_network():
  """
  Get notes
  Generates input and putput sequences
  Create model
  TRains the model for the given epochs
  """

  epochs = 200

  notes_data = get_notes()
  print('Notes processed')

  n_vocab = len(set(notes_data))
  print('Vocab generated')

  network_in, network_out = sequences(notes_data, n_vocab)
  print('Input and Output processed')

  model = create_network(network_in, n_vocab)
  print('Model created')
  #return model
  print('Training in progress')
  train(model,network_in, network_out, epochs)
  print('Training completed')

  return model, network_in, n_vocab

In [ ]:
notes_data = get_notes()
print(f"Extracted {len(notes_data)} notes/chords.")
print("First 10 entries:", notes_data[:10])

Extracted 3133 notes/chords.
First 10 entries: ['G4', 'F#4', 'G4', 'F#4', 'E4', 'G4', 'F#4', 'G4', 'F#4', 'G4']


In [ ]:
model, network_input, n_vocab = train_network()

Notes processed
Vocab generated
Input and Output processed


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model created
Training in progress
Epoch 1/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 4.8138
Epoch 1: loss improved from inf to 4.69126, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - loss: 4.8113
Epoch 2/200
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 4.6092
Epoch 2: loss improved from 4.69126 to 4.58000, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 4.6080
Epoch 3/200
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 4.4763
Epoch 3: loss improved from 4.58000 to 4.46982, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 4.4760
Epoch 4/200
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 4.3524
Epoch 4: loss improved from 4.46982 to 4.35566, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━

In [ ]:
model = create_network(network_input, n_vocab)
model.load_weights("/content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras")
model.save("/content/drive/MyDrive/University/AI2Arts/model_complete.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 15 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Note Generation


In [ ]:
def get_inputSequence(notes, pitchnames, n_vocab):
  """ Prepare the sequence used by the Neural Network """
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  sequence_length = 100
  network_input = []
  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

  network_input = np.reshape(network_input, (len(network_input), 100, 1))

  return (network_input)

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
  """ Generate notes from the neural network based on a sequence of notes """

  start = np.random.randint(0, len(network_input)-1)

  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

  pattern = list(network_input[start])
  prediction_output = []

  print('Generating notes....')

  for note_index in range(250):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append([index])
    pettern = pattern[1:len(pattern)]

  print('Notes Generated...')
  return prediction_output

In [ ]:
def create_midi(prediction_output):
  """ Convert the output from the preduction to notes and create a midi file from the notes """
  offset = 0
  output_notes = []

  for pattern in prediction_output:
    if('.' in pattern) or pattern.isdigit():
      notes_in_chord = pattern.split('.')
      notes = []
      for current_note in notes_in_chord:
        new_note = note.Note(int(current_note))
        new:note.storeInstrument = instrument.Guitar()
        notes.append(new_note)
      new_chord = chord.Chord(notes)
      new_chord.offset = offset
      output_notes.append(new_chord)
    else:
          new_note = note.Note(pattern)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)

    offset += 0.5

  midi_stream = stream.Stream(output_notes)

  print('Saving Output file as midi...')
  midi_stream.write('midi', fp='test_output.mid')

In [ ]:
def generate():
  """Generate a piano midi file"""
  path = '/content/drive/My Drive/University/AI2Arts/data'
  with open(os.path.join(path, 'notes'), 'rb') as filepath:
    notes = pickle.load(filepath)

  pitchnames = sorted(set(item for item in notes))
  n_vocab = len(set(notes))

  print('Initating music generation process...')

  network_input = get_inputSequence(notes, pitchnames, n_vocab)
  normalized_input = network_input / float(n_vocab)
  model = create_network(normalized_input, n_vocab)
  model.summary()
  print('Loading model weights')
  model.load_weights('/content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras')
  print('Model Loaded')
  prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
  create_midi(prediction_output)

  print('Song Generated')

In [ ]:
generate()

Initating music generation process...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 256)       │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 512)       │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 178)            │        45,746 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 178)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,738,098 (10.45 MB)

 Trainable params: 2,738,098 (10.45 MB)

 Non-trainable params: 0 (0.00 B)

Loading model weights


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 15 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model Loaded
Generating notes....
Notes Generated...
Saving Output file as midi...
Song Generated


## Play MIDI

In [ ]:
import pygame

def play_music(music_file):
  """ Stream music with mixer.music module in blocking manner.
  This will stream the sound from disk while playing """

  clock = pygame.time.clock()
  try:
    pygame.mixer.music.load(music_file)
    print("Music file %s loaded!" % music_file)
  except pygame.error:
    print("File %s not found! (%s)"% (music_file, pygame.get_error()))
    return
  pygame.mixer.music.play()
  while pygame.mixer.music.get_busy():
    clock.tick(30)

In [ ]:
def play_midi(midi_file):
  freq = 44100
  bitsize = -16
  channels = 2
  buffer = 1024
  pygame.mixer.init(freq, bitsize, channels, buffer)

  pygame.mixer.music.set_volume(0.8)

  try:
    play_music(midi_file)
  except KeyboardInterrupt:
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

In [ ]:
from google.colab import files
files.download("test_output.mid")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
play_midi('test_output.mid')

error: ALSA: Couldn't open audio device: No such file or directory